# Introdução 

Neste notebook, faremos algumas visualizações do dataset cora.

# 1- Dataset no formato tabular

A representação em grafos do dataset cora também está disponível no formato csv através da página do grupo <a href='https://linqs.soe.ucsc.edu/data'> LINQS </a>. No entanto, não é necessário realizar o download, pois ele já consta na pasta <a href='cora.csv'>cora.csv</a> do diretório atual deste notebook. Para tanto, vamos referenciar o arquivo <code>cora.content</code> (matriz de características e classes) e <code>cora.cites</code> (relações de citação).

In [ ]:
path_content = 'cora.csv/cora.content'
path_cites = 'cora.csv/cora.cites'

Em seguida, vamos importar as tabela a partir do <code>pandas</code>

In [ ]:
import pandas as pd


#Carrega citações
cites = pd.read_csv(path_cites, sep='\t', header=None, names=['target','source'])

#Nome das colunas referentes às palavras
feature_names = [f'P_{_}' for _ in range(1433)]

#Define o nome de todas as colunas para a tabela de características e classificações
column_names = ['id'] + feature_names + ['label']

#Carrega características e classificações
content = pd.read_csv(path_content, sep='\t', header=None, names=column_names)

#Ordena a tabela de acordo com o id do artigo
content = content.sort_values('id')


Agora, vamos visualizar a tabela contendo as características e classificações:

In [ ]:
content

Agora, vamos visualizar a tabela contendo as citações. Nesse caso, vamos transpô-la para melhorar a visualização:

In [ ]:
cites.T

# 2- Análise de PDFs

Segundo o <a fref= 'Introduction to the theory of computation_third edition - Michael Sipser.pdf'> artigo </a> que referencia o dataset cora que empregamos até o momento, durante o preprocessamento das publicações do dataset, houve a remoção de _stopwords_ e normalização textual. 

>__stopwords__: palavras mais comuns em uma língua. Por exemplo, no inglês, poderíamos citar: “the”, “is”, “in”, “for”, “where”, “when”, “to”, “at” etc.

Para tanto, vamos carregar um pdf:

In [ ]:
#!conda install -c conda-forge pdfplumber -y

In [ ]:
import pdfplumber

#Carrega o PDF
pdf = pdfplumber.open('a.pdf')
#Verifica a quantidade de páginas
num_pages = len(pdf.pages)

#String para o texto do PDF
text_from_pdf = ''
#Para cada página, extrai o texto 
for _ in range(num_pages):
    text_from_pdf +=  pdf.pages[_].extract_text()


Primeiro, precisamos realizar a tokenização do texto. Vamos fazer isso com o módulo spaCy:

In [ ]:
#!conda install -c conda-forge spacy -y

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
example_sent = """This is a sample sentence,
                  showing off the stop words filtration."""
 
stop_words = set(stopwords.words('english'))
 
word_tokens = word_tokenize(example_sent)
 
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
 
filtered_sentence = []

Agora, vamos remover as _stopwords_:

Como é possível pereceber, há muitos tokens como pontuações, dígitos e caracteres de escape. Vamos retirá-los:

In [ ]:
import string

#Removendo pontuações
filtered_sentence = [_ for _ in filtered_sentence if _ not in string.punctuation]
#Removendo dígitos 
filtered_sentence = [_ for _ in filtered_sentence if _ not in string.digits]


In [ ]:
import pandas as pd

pd.Series(filtered_sentence).value_counts().to_frame()[:20]


Inicializando o dataset:

In [ ]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='.', name='Cora')

num_node_features = dataset.num_node_features
embedding_size = 16
num_classes = dataset.num_classes
data = dataset[0]

Implementando o modelo:

In [ ]:
from torch.nn import Module
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, ChebConv, GATConv, SGConv

#x = F.dropout(x, training=self.training) #Dropout

class GCN(Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = SGConv(in_channels, 16, 2)
        self.conv2 = SGConv(16, out_channels, 2)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) 
        x = F.relu(x) 
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index) 
        return F.log_softmax(x, dim=1)

'''
#O modelo herda a classe Module (base para qualquer modelo de nn)
class GCN(Module):
    def __init__(self, in_channels, out_channels):

        super().__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        
        x = self.conv1(x, edge_index) 
        x = F.relu(x)
        #x = F.dropout(x, training=self.training) #Dropout
        x = self.conv2(x, edge_index) 

        return F.log_softmax(x, dim=1)
'''

Escolha do dispositivo

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

Transferindo modelo e dataset para o dispositivo

In [ ]:
model = GCN(num_node_features, num_classes).to(device)
data = dataset[0].to(device)

Escolhendo a função de custo e o otimizador:

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4) 

#loss_function = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4) 
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4) 



Definindo a função de treinamento:

In [ ]:
def train():
    optimizer.zero_grad() #Limpa o gradiente
    out = model(data.x, data.edge_index) # Realiza a propagação
    loss = loss_function(out[data.train_mask], data.y[data.train_mask]) # Computa o loss apenas do conjunto de treinamento
    loss.backward()  # Calcula o gradiente 
    optimizer.step() # Atualiza os parâmetros do modelo com base no gradiente
        
    return loss

Definindo a função de avaliação:

In [ ]:
def aval_train():
    pred = model(data.x, data.edge_index).argmax(dim=1) # Realiza predições
    correct = (pred[data.val_mask] == data.y[data.val_mask]).sum() # Soma os acertos totais
    acc = int(correct) / int(data.val_mask.sum()) # Calcula a acurácia acertos / total
    return acc

Treinamento:

In [ ]:
import numpy as np

num_epochs = 200

accuracies = []
losses = []

epochs = np.arange(1,num_epochs+1)

for epoch in epochs:
    loss = train() #Realiza o treinamento
    accuracy = aval_train() #Calcula a acuráciapNLLLoss()
    losses.append(float(loss)) #Anexa o loss atual a lista de losses
    accuracies.append(accuracy) # Anexa a acurácia atual na lista de acuracias
    if epoch%50 == 0: print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acurácia: {accuracy}') #Imprima o status a cada 50 epochs

Plotando os resultados:

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib notebook

fig, ax = plt.subplots(figsize=(7,7))
ax.plot(epochs, losses, color='red')
ax.plot(epochs, accuracies, color='blue')
ax.set(xlabel='EPOCH')